# HeadHunt

We're going to teach a program what kinds of jobs we like, using labeled Gmail messages as a training source.

Suggested fields:
timestamp type source raw text label

# Where was I?

To recap, we have:

* Downloaded _all_ of our emails, w/ their labels included
* Extracted their underlying text, for use as a Bag of Words
* Classified good job offers from bad job offers

Now, our challenge is to:

* Classify what whether something _is_ a job offer
* If it is, run it through the job-offer classifier to decide whether it's a good or bad offer.

To accomplish the first, we simply need an boolean index into our 
dataset that says whether something is a job-offer (True) or 
not (False). The challenge is our labels live as a list inside the downloaded Gmail Message resource, and we'd need to perform a boolean `contains` operation on each message. We could formalize our problem as:

    ???
    
The second is a bit easier to solve. We have a tree of labels, starting with a root of no label at all. Each non-leaf node of the tree will have an associated classifier, which assigns likelihoods to each of its children. We then simply have to run `predict` at each level of the tree, and follow the most likely path(s).

In [112]:
%matplotlib inline
import pickle 
import sys

MATCHWELL = '../matchwell'
if MATCHWELL not in sys.path:
    sys.path.insert(0, MATCHWELL)

import pandas as pd
%load_ext autoreload
%autoreload 2
from matchwell import models, gmail, measure, util
from importlib import reload


# Constants
POS_LABEL = 'job-offers/yes'
NEG_LABEL = 'job-offers/no'
DATAFILE = './matchwell.h5'

# Load DataFrame; 'all' or 'emails' are the choices
# all: The full set of downloaded emails & text messages
# emails: A pre-processed set of emails for training
df = pd.read_hdf(DATAFILE, 'emails') 

print("Loaded %d samples" % len(df))

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
Loaded 13950 samples


In [118]:
# from importlib import reload
# reload(gmail)
# Update
gs = gmail.GmailSource()
# msgs = gs._extract(query='after:' + util.newest(df, format=True))
ndf = gs.pull(newer_than=util.newest(df, format=True))

Retrieving messages newer than 2016/06/29
Retrieving messages newer than after:2016/06/29
Retrieving messages with *any* label
Retrieved 103 message IDs in 0.022 seconds
Downloaded 100 emails
Retrieved another 103 message IDs in 0.006 seconds
Downloaded 100 emails
Retrieved another 103 message IDs in 0.007 seconds
Downloaded 100 emails
Retrieved another 103 message IDs in 0.006 seconds
Downloaded 100 emails
Retrieved another 103 message IDs in 0.004 seconds
Downloaded 100 emails
Retrieved another 103 message IDs in 0.005 seconds
Downloaded 100 emails
Retrieved another 103 message IDs in 0.006 seconds
Downloaded 100 emails
Retrieved another 103 message IDs in 0.010 seconds
Downloaded 100 emails
Retrieved another 103 message IDs in 0.005 seconds
Downloaded 100 emails
Retrieved another 72 message IDs in 0.009 seconds
Downloaded 72 emails
Retrieved 999 total message IDs in 4.777 seconds


In [123]:
mergedf = pd.concat([df, ndf])
assert len(mergedf) == len(df) + len(ndf)

In [ ]:
# Save
# df.to_hdf(DATAFILE, 'emails')

In [22]:
X, y = df['text'], df['labels']

NameError: name 'table' is not defined